<a href="https://colab.research.google.com/github/AnastasiaBrinati/experiments-Flower/blob/main/Endpoint1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MODELLO CENTRALIZZATO SU ENDPOINT 1:

imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf

from tensorflow import keras
from keras import layers
from tensorflow.keras.callbacks import EarlyStopping

!pip install -q -U keras-tuner
import keras_tuner as kt

# Use seaborn for pairplot.
!pip install datasets

dataset

In [104]:
from datasets import load_dataset
dataset = load_dataset("anastasiafrosted/my_sequences_endpoint1", download_mode="force_redownload")

README.md:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/7.71M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/104837 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/19297 [00:00<?, ? examples/s]

In [168]:
training_set = pd.DataFrame(dataset['train'])
test_set = pd.DataFrame(dataset['test'])

In [169]:
x_train = training_set.copy()
x_test = test_set.copy()

In [170]:
# Pop two target features into y_train and y_test
y_train = x_train[['avg_execution_time_target', 'avg_scheduling_time_target']]
y_test = x_test[['avg_execution_time_target', 'avg_scheduling_time_target']]

# Save aside timestamps
x_train_timestamps = x_train['timestamp_seq']
x_test_timestamps = x_test['timestamp_seq']

y_train_timestamps = x_train['timestamp_target']
y_test_timestamps = x_test['timestamp_target']

# Drop the targets from predictors
x_train = x_train.drop(['avg_execution_time_target', 'avg_scheduling_time_target', 'timestamp_seq', 'timestamp_target'], axis=1)
x_test = x_test.drop(['avg_execution_time_target', 'avg_scheduling_time_target', 'timestamp_seq', 'timestamp_target'], axis=1)

columns = x_train.columns.tolist()
targets = ['avg_execution_time_target', 'avg_scheduling_time_target']

In [171]:
# Extract the first value of each list
first_values = np.array([timestamps[0] for timestamps in x_test_timestamps[:-1]])
print(len(first_values))
last_sequence = x_test_timestamps[-1:][1:]
timestamps = np.concatenate([first_values, last_sequence])

19296


In [172]:
print(timestamps)

['2023-06-12:0:0' '2023-06-12:0:1' '2023-06-12:0:10' ... '2023-07-03:1:48'
 '2023-07-03:1:49' '2023-07-03:1:5']


In [173]:
# Convert each column to a 2D NumPy array (samples x timesteps)
x_train_as_arrays = [np.array(x_train[col].to_list()) for col in columns]
x_test_as_arrays = [np.array(x_test[col].to_list()) for col in columns]

y_train_as_arrays = [np.array(y_train[col].to_list()) for col in targets]
y_test_as_arrays = [np.array(y_test[col].to_list()) for col in targets]

In [174]:
# Stack columns to create a 3D array (samples, timesteps, features)
x_train = np.stack(x_train_as_arrays, axis=-1)
x_test = np.stack(x_test_as_arrays, axis=-1)

y_train = np.stack(y_train_as_arrays, axis=-1)
y_test = np.stack(y_test_as_arrays, axis=-1)

# Verify the shape of the final 3D array
print(f"x_train shape: {x_train.shape}")  # Expected: (104837, timesteps, 14)
print(f"x_test shape: {x_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

x_train shape: (104837, 7, 11)
x_test shape: (19297, 7, 11)
y_train shape: (104837, 7, 2)
y_test shape: (19297, 7, 2)


model

In [31]:
# Check GPU availability
if tf.test.gpu_device_name():
    print(f"GPU disponibile: {tf.test.gpu_device_name()}")
else:
    print("GPU non disponibile. L'esecuzione potrebbe essere più lenta.")

GPU non disponibile. L'esecuzione potrebbe essere più lenta.


In [32]:
# Class for the LSTM model
class Lstm:
    def __init__(self, learning_rate, sequence_length, num_features):
        self.learning_rate = learning_rate
        self.sequence_length = sequence_length  # Number of timesteps in each sequence
        self.num_features = num_features  # Number of features per timestep

        # Define the LSTM-based model
        self.model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(self.sequence_length, self.num_features)),  # Input shape for LSTM
            tf.keras.layers.LSTM(50, activation='relu', return_sequences=True),  # First LSTM layer
            tf.keras.layers.LSTM(25, activation='relu', return_sequences=True),  # Second LSTM layer for sequence output
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(10, activation='relu')),  # Fully connected layer for each timestep
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(2))  # Output layer for each timestep
        ])

        # Define the loss function and optimizer
        self.loss_function = tf.keras.losses.MeanSquaredError()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)

    def compile(self):
        # Compile the model with optimizer, loss, and additional metrics
        self.model.compile(optimizer=self.optimizer, loss=self.loss_function,
                           metrics=["cosine_similarity", tf.keras.metrics.MeanAbsoluteError()])

    def get_model(self):
        return self.model


In [33]:
# Usage example
learning_rate = 0.001
sequence_length = 7  # Number of timesteps in each sequence
num_features = 11  # Number of features per timestep

# Instantiate and compile the model
lstm_model = Lstm(learning_rate, sequence_length, num_features)
lstm_model.compile()

# Summary of the model
print(lstm_model.get_model().summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                        │ (None, 7, 50)               │          12,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 7, 25)               │           7,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_4 (TimeDistributed) │ (None, 7, 10)               │             260 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_5 (TimeDistributed) │ (None, 7, 2)                │              22 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,282 (79.23 KB)

 Trainable params: 20,282 (79.23 KB)

 Non-trainable params: 0 (0.00 B)

None


Training

In [175]:
# Train the model
history = lstm_model.get_model().fit(
    x_train, y_train,
    epochs=10,  # Adjust epochs as needed
    batch_size=16,  # Batch size for training
    validation_split=0.2  # Use 20% of the data for validation
)

Epoch 1/10
5242/5242 ━━━━━━━━━━━━━━━━━━━━ 73s 14ms/step - cosine_similarity: 0.9668 - loss: 0.9148 - mean_absolute_error: 0.0335 - val_cosine_similarity: 0.9756 - val_loss: 0.0052 - val_mean_absolute_error: 0.0231
Epoch 2/10
5242/5242 ━━━━━━━━━━━━━━━━━━━━ 76s 13ms/step - cosine_similarity: 0.9542 - loss: 0.7205 - mean_absolute_error: 0.0327 - val_cosine_similarity: 0.9880 - val_loss: 0.0055 - val_mean_absolute_error: 0.0274
Epoch 3/10
5242/5242 ━━━━━━━━━━━━━━━━━━━━ 72s 14ms/step - cosine_similarity: 0.9650 - loss: 0.8988 - mean_absolute_error: 0.0377 - val_cosine_similarity: 0.9863 - val_loss: 0.0051 - val_mean_absolute_error: 0.0146
Epoch 4/10
5242/5242 ━━━━━━━━━━━━━━━━━━━━ 75s 12ms/step - cosine_similarity: 0.9533 - loss: 0.6932 - mean_absolute_error: 0.0327 - val_cosine_similarity: 0.9947 - val_loss: 0.0053 - val_mean_absolute_error: 0.0209
Epoch 5/10
5242/5242 ━━━━━━━━━━━━━━━━━━━━ 82s 12ms/step - cosine_similarity: 0.9628 - loss: 0.8259 - mean_absolute_error: 0.0338 - val_cosine_si

graph loss

In [ ]:
# Plot training history
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

Evaluation

In [ ]:
# Evaluate the model on test data
test_loss, *test_metrics = lstm_model.get_model().evaluate(x_test, y_test, verbose=1)
print(f"Test Loss: {test_loss}")
for i, metric in enumerate(test_metrics):
    print(f"Test Metric {i+1}: {metric}")

Prediction

In [176]:
# Generate predictions
y_pred = lstm_model.get_model().predict(x_test, verbose=1)

print(f"Predictions shape: {y_pred.shape}")
print(f"First prediction: {y_pred[0]}")


604/604 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step
Predictions shape: (19297, 7, 2)
First prediction: [[0.044 0.004]
 [0.044 0.004]
 [0.044 0.004]
 [0.044 0.004]
 [0.044 0.004]
 [0.044 0.004]
 [0.044 0.004]]


In [177]:
# Step 1: Prendi la prima target feature (feature 1)
y_pred_feature1 = y_pred[:, :, 0]  # Shape: (num_samples, timesteps)
y_test_feature1 = y_test[:, :, 0]  # Shape: (num_samples, timesteps)

# Step 2: Prendi il primo valore per ogni sequenza eccetto l'ultima
first_values_pred = np.array([seq[0] for seq in y_pred_feature1[:-1]])
first_values_actual = np.array([seq[0] for seq in y_test_feature1[:-1]])

# Step 3: Aggiungi l'intera ultima sequenza
last_sequence_pred = y_pred_feature1[-1]  # L'ultima sequenza completa
last_sequence_actual = y_test_feature1[-1]  # L'ultima sequenza completa

# Step 4: Combina i valori in un unico array
new_array_pred = np.concatenate([first_values_pred, last_sequence_pred])
new_array_actual = np.concatenate([first_values_actual, last_sequence_actual])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Grafico
plt.figure(figsize=(12, 6))
plt.plot(timestamps[15000:], new_array_pred[15000:-7], label='Actual Feature 1', color='blue')
plt.plot(timestamps[15000:], new_array_actual[15000:-7], label='Predicted Feature 1', linestyle='dashed', color='orange')

# Personalizzazione del grafico
plt.title("Evoluzione nel tempo della Target Feature 1")
plt.xlabel("Timestamp")
plt.ylabel("Valore della Target Feature 1")
plt.xticks(rotation=45)  # Rotazione dei timestamp per leggibilità
plt.legend()
plt.grid()
plt.tight_layout()  # Per evitare sovrapposizioni
plt.show()
